# Text Summarization
#### Для задания будем использовать MBART

In [ ]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [ ]:
!ls

## mBART huggingface
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210
* Model: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [13]:
import json
import torch
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, MBartForConditionalGeneration

In [14]:



def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size


def predict(
    model_name,
    test_file,
    predictions_file,
    targets_file,
    max_source_tokens_count=600,
    max_target_tokens_count=160,
    use_cuda=True,
    batch_size=4
):
    inputs = []
    targets = []
    with open(test_file, "r", encoding='utf8') as r:
        for line in r:
            record = json.loads(line)
            inputs.append(record["text"])
            targets.append(record["summary"].replace("\n", " "))
            
    inputs = inputs[:20]
    targets = targets[:20]
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    device = torch.device("cuda:0") if use_cuda else torch.device("cpu")
    model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)
    predictions = []
    for batch in tqdm(gen_batch(inputs, batch_size)):
        input_ids = tokenizer.prepare_seq2seq_batch(
            batch,
            src_lang="en_XX",
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=max_source_tokens_count
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=max_target_tokens_count + 2,
            no_repeat_ngram_size=3,
            num_beams=5,
            top_k=0
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        predictions.extend(summaries)
    with open(predictions_file, "w", encoding='utf8') as w:
        for p in predictions:
            w.write(p.strip() + "\n")
    with open(targets_file, "w", encoding='utf8') as w:
        for t in targets:
            w.write(t.strip() + "\n")
    with open('n_input.txt', "w", encoding='utf8') as w:
        for t in inputs:
            w.write(t.strip() + "\n")

predict("IlyaGusev/mbart_ru_sum_gazeta", "gazeta_test.txt", "predictions.txt", "targets.txt")

0it [00:00, ?it/s]

In [16]:
def get_different(n=0):
    with open('n_input.txt', "r", encoding='utf8') as r:
        data = r.readlines()
        print(f"Inint: {data[n]}")
    with open('targets.txt', "r", encoding='utf8') as r:
        data = r.readlines()
        print(f"Target: {data[n]}")
    with open('predictions.txt', "r", encoding='utf8') as r:
        data = r.readlines()
        print(f"Predict: {data[n]}")

In [17]:
get_different(4)

Inint: Депутат Верховной рады от партии «Слуга народа» Елизавета Богуцкая раскритиковала законопроект о выплате пенсий украинцам, проживающим на территориях Донбасса. По ее мнению, эти люди могут придерживаться пророссийских взглядов. «Кто будет определять, какие взгляды у тех пенсионеров? Поддерживают они Россию или нет? Мы им будем выплачивать? Мы сейчас плюем на всех переселенцев, которые сейчас нуждаются в жилье, в выплатах здесь и ничего не получают», — заявила Богуцкая. Так она прокомментировала закон о праве на пенсию для жителей Донбасса. 5 февраля Верховная рада отправила проект на повторное первое чтение. Согласно документу, отправленному на доработку, должна быть упрощена процедура доказательства права на выплату пенсий для тех, кто был вынужден покинуть свое место жительства из-за вооруженного конфликта. К тому же законопроект предусматривал установление особого режима и порядок получения выплат для пенсионеров, которые остались проживать на территориях Донбасса. Кроме того